In [1]:
import math
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline


import time as t

In [2]:
import os
import gym
from gym import spaces, envs
import argparse
import numpy as np
import itertools
import time
from builtins import input
import random
#import randomizer

#from randomizer.wrappers import RandomizedEnvWrapper
from mujoco_py.modder import TextureModder, MaterialModder
import cv2

from multiprocessing import Pool

In [3]:
#Function to create the enviorments

def CreateEnviorments( Environment = "Gen3SidewaysFold-v0", Nenv = 10 ):
    env_list = []
    for i in range(Nenv):
        env_list.append(envs.make("Gen3SidewaysFold-v0"))
    return env_list

In [4]:
def SimulateAndModParams(ParameterToChange,ParameterToChangeValues,Actions,render):
    
    env=envs.make("Gen3SidewaysFold-v0")
    render_mode = "human"
    observations = []
    infos = []
    numItr = 100
    #print("Actions",Actions)
    print("Actions len",len(Actions))
    print("Actions 1",Actions[0])
    
    obs = env.reset()


    env.Reset_Save_Data_On_Environment()
    
    env.env.Change_Mujoco_Parameters(ParameterToChange,ParameterToChangeValues)

    for i in range (len(Actions)):
        #print(i)
        actionRescaled = Actions[i]
        #print("Action Rescaled",actionRescaled)
        

        if render: 
            env.render(mode=render_mode)

        obs, reward, done, info = env.step(actionRescaled)
        env.Save_Data_On_Environment()
        
    return env


In [ ]:
#test Multiprocess Simulation


#ctrlrange', 'forcerange', 'gainprm', 'mass', 'inertia'
#env.env.Change_Mujoco_Parameters(["mass"],[100]*7)


NumberOfTest = 4
HalfOfTest = int(NumberOfTest*0.5)

#envList = CreateEnviorments(Nenv = NumberOfTest)

ParameterToChangeList = [["mass"]]*HalfOfTest + [["inertia"]]*(NumberOfTest-HalfOfTest)
ParameterToChangeValuesList = [[100]*7]*HalfOfTest + [[100,100,100]*7]*(NumberOfTest-HalfOfTest)

actionRescaled1 = [[0.1,0.1,0.1,1]]*100
actionRescaled2 = [[-0.1,0.1,0.1,0]]*100
actionRescaled3 = [[0.1,-0.1,0.1,1]]*100
actionRescaled4 = [[0.1,0.1,-0.1,0]]*100

renderList = [True]*(NumberOfTest-1)
renderList = [True]+renderList

Actions = actionRescaled1 + actionRescaled2 + actionRescaled3 + actionRescaled4


#Creation of the parameters
ParametersList = []
for i in range(NumberOfTest):
    ParameterToChange = ParameterToChangeList.pop(0)
    ParameterToChangeValues = ParameterToChangeValuesList.pop(0)
    
    #env = envList[i]
    render = renderList.pop(0)
    
    Parameters = [ParameterToChange,ParameterToChangeValues,Actions,render]
    #SimulateAndModParams(env,ParameterToChange,ParameterToChangeValues,Actions,render)
    #print(Parameters)
    ParametersList.append(Parameters)

#creation of the multiprocesses
pool = Pool(processes=NumberOfTest)
print("Starting Simunlation")
envList = pool.starmap(SimulateAndModParams,ParametersList)

sleep(4)


Starting Simunlation
Creating window glfw
Creating window glfw
Creating window glfw
Creating window glfw


In [6]:
#Create a experiment env
env = envs.make("Gen3SidewaysFold-v0")

#Initially parameters of the urdf

#Make maxvels closer to the reality
#both have to be modified
env.max_vel = [168,151,128,94,210,48,189]
env.original_parameters_df ["max_vel"]=env.max_vel
print(env.original_parameters_df["max_vel"])
print(env.original_parameters_df)


print('observation space:', env.observation_space) #states, There is only 1 state constant
env.update_parameters_to_modify(["mass","max_vel","kp","ki","kd","force_x_one","Ixx","Iyy","Izz"])
#env.update_parameters_to_modify_base(["mass_base","Ixx_base","Iyy_base","Izz_base"])
print('action space:', env.action_space) #parameters, number of parameters choose to tune, continuous
print('original action:', env.action_original()) #parameters, number of parameters choose to tune, continuous




Creating window glfw


AttributeError: 'Gen3SidewaysFoldEnv' object has no attribute 'original_parameters_df'

In [ ]:
#Cross Entrophy Method, to choose the weights

# In my case where only 1 action,and that it's apply the parameters do another step doesn't change anything due to the state doesn't vary
# For this reason max_t and gama doesn't make sense, so I set them to max_t = 1 and gamma to 0
def cem_no_net(n_iterations=700, max_t=1, gamma=0.0, print_every=50, pop_size=env.action_space, elite_frac=10/(env.action_space*2), sigma=0.3,sigma_reduction_every_print = 0.65, per_one = True ):
    """PyTorch implementation of the cross-entropy method.
        
    Params
    ======
        n_iterations (int): maximum number of training iterations
        max_t (int): maximum number of timesteps per episode
        gamma (float): discount rate
        print_every (int): how often to print average score (over last 100 episodes)
        pop_size (int): size of population at each iteration
        elite_frac (float): percentage of top performers to use in update
        sigma (float): standard deviation of additive noise
        per_one (boolean): to determine if the output is in per one or not
    """
    #Numbers of elements that you keep as the better ones
    n_elite=int(pop_size*elite_frac)
    
    #scores doble end queee , from iterations size * 0.1
    scores_deque = deque(maxlen=int(n_iterations*0.1))
    #intial scores empty
    scores = []
    #Save actions to see how they evolve
    best_actions = []
    #Select a seed to make the results the same every test, not depending on the seed
    np.random.seed(0)
    #Initial best weights, are from 0 to 1, it's good to be small the weights, but they should be different from 0.
    # small to avoid overfiting , different from 0 to update them
    
    if (per_one == True):
        best_weight = sigma*np.random.randn(env.action_space)
        original_action = np.array(env.action_original())
    else:
        best_weight = np.add(sigma*np.random.randn(env.action_space),env.action_original())

    #Each iteration, modify  + (from 0 to 1) the best weight randomly
    #Computes the reward with these weights
    #Sort the reward to get the best ones
    # Save the best weights
    # the Best weight it's the mean of the best one
    #compute the main reward of the main best rewards ones
    #this it's show to evalute how good its
    
    for i_iteration in range(1, n_iterations+1):
        
        #Generate new population weights, as a mutation of the best weight to test them
        weights_pop = [best_weight + (sigma*np.random.randn(env.action_space)) for i in range(pop_size)]
        
        #Compute the parameters and obtain the rewards for each of them
        #print("iteration "+str(i_iteration))
        if (per_one == True):
            rewards=[]
            for weights in weights_pop:
                #print("New weights")
                #print(weights)
                #t.sleep(1000)
                action=np.add(np.multiply(weights,original_action),original_action)
                rewards.append( env.step_tcp_rishabh_joints_offset(action,Experiment_time=1.22) )
            rewards = np.array(rewards)
        else:
            rewards=[]
            for weights in weights_pop:
                rewards.append(env.step_tcp_rishabh_joints_offset(weights,Experiment_time=1.22) )
            rewards = np.array(rewards)
        print("rewards" + str(i_iteration))
        print(rewards)
        #print("\n")
        
        #Sort the rewards to obtain the best ones
        elite_idxs = rewards.argsort()[-n_elite:]
        elite_weights = [weights_pop[i] for i in elite_idxs]
        
        #Set the best weight as the mean of the best ones 
       
        best_weight = np.array(elite_weights).mean(axis=0)
        
        #Get the reward with this new weight
        
        if (per_one == True):
            action = np.add(np.multiply(best_weight,original_action),original_action)
            #print("action vel")
            #print(action[8])
            #print("action kp")
            #print(action[15])
            #time.sleep(1000)
            best_actions.append(action)
            reward = env.step_tcp_rishabh_joints_offset(action,Experiment_time=1.22)
            print("reward")
            print(reward)
            print("\n")
        else:
            best_actions.append(best_weight)
            reward = env.step_tcp_rishabh_joints_offset(best_weight,Experiment_time=1.22)
        scores_deque.append(reward)
        scores.append(reward)
        
        #save the check point
        env.save_parameters("./Parameters_train_tcp_euc_rishabh_joint_offset_base.xlsx")
        
        if i_iteration % print_every == 0:
            sigma = sigma * sigma_reduction_every_print
            print('Episode {}\tAverage Score: {:.2f}'.format(i_iteration, np.mean(scores_deque)))

        if np.mean(scores_deque)>=0.0:
            print('\nEnvironment solved in {:d} iterations!\tAverage Score: {:.2f}'.format(i_iteration-n_iterations*0.1, np.mean(scores_deque)))
            break
    return scores,best_actions


In [ ]:
#Execute the cross entrophy method with default Values
#scores = cem()


#To don't ask the GPU as much reduce the pop_size, it's the amount of elemts try
scores,best_actions = cem_no_net()
# 
# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()
    

In [ ]:

# plot the last scores zoom
fig = plt.figure()
zoom= 300
ax = fig.add_subplot(111)
plt.plot(np.arange(1, zoom+1), scores[-zoom:])
plt.ylabel('Score zoom')
plt.xlabel('Episode #')
plt.show()

In [ ]:
# plot actions
best_actions_np = np.array(best_actions)
for joint in range(7):
    for i in range(len(env.parameters_to_modify)) :
        parameter = env.parameters_to_modify[i]
        figures = plt.figure()
        ax = fig.add_subplot(111)
        plt.plot(np.arange(1, best_actions_np.shape[0]+1), best_actions_np[:,joint+i*7])
        plt.ylabel(parameter+" Joint"+str(joint))
        plt.xlabel('Episode #')
        plt.show()

In [ ]:
# load the weights from file
# Not working know


#state = env.reset()
env = env_pybullet_kin_gen3(no_zeros = True,Excels_path_Okay_tcp_list = ["Tcp_Trajectori_19_1.xlsx","Tcp_Trajectori_39_1.xlsx","Tcp_Trajectori_59_1.xlsx"],\
                            Actions_offsets_Numpy_files = ["acs_19_tcp.npy","acs_39_tcp.npy","acs_59_tcp.npy"],time_step=0.02,home_angles = [-0.207226464676801,1.5689993219813,-1.01515387451347,-2.45271819663908,2.00795352004673,1.91098991659003,-0.831045149646278])
env.robot.visual_inspection = True

#Make maxvels closer to the reality
env.max_vel = [168,151,128,94,210,48,189]
env.original_parameters_df ["max_vel"]=env.max_vel
env.modified_parameters_df ["max_vel"]=[168,151,128,94,210,48,189]

env.update_parameters_to_modify(["mass","max_vel","kp","ki","kd","force_x_one","Ixx","Iyy","Izz","damping"])
#env.update_parameters_to_modify_base(["mass_base","Ixx_base","Iyy_base","Izz_base"])
env.robot.visual_inspection = True
env.modified_parameters_df = env.create_df_from_Excel("./Parameters_train_tcp_euc_rishabh_joint_offset_base.xlsx")


t.sleep(0.02)
action = env.action_modified()
action = np.array(action)
print('original action:', env.action_original()) #parameters, number of parameters choose to tune, continuous
print("trained",action)
reward = env.step_tcp_rishabh_joints_offset(action,Experiment_time=1.22)
print("reward")
print(reward)



In [ ]:
#Convert to excel
a = env.df_avg.to_numpy()
print(a[:,5])
env.df_avg.to_excel("./Train_parameters_result_tcp_euc_rishabh_joint_offset_base_19.xlsx")

In [ ]:
env.original_parameters_df

In [ ]:
#Original ERROR
# load the weights from file
# Not working know


#state = env.reset()
env = env_pybullet_kin_gen3(no_zeros = True,Excels_path_Okay_tcp_list = ["Tcp_Trajectori_19_1.xlsx","Tcp_Trajectori_39_1.xlsx","Tcp_Trajectori_59_1.xlsx"],\
                            Actions_offsets_Numpy_files = ["acs_19_tcp.npy","acs_39_tcp.npy","acs_59_tcp.npy"],time_step=0.02,home_angles = [-0.207226464676801,1.5689993219813,-1.01515387451347,-2.45271819663908,2.00795352004673,1.91098991659003,-0.831045149646278])
#Make maxvels closer to the reality
env.max_vel = [168,151,128,94,210,48,189]
env.original_parameters_df ["max_vel"]=env.max_vel
env.modified_parameters_df ["max_vel"]=[168,151,128,94,210,48,189]

env.update_parameters_to_modify(["mass","max_vel","kp","ki","kd","force_x_one","Ixx","Iyy","Izz"])
#env.update_parameters_to_modify_base(["mass_base","Ixx_base","Iyy_base","Izz_base"])
env.robot.visual_inspection = True
env.modified_parameters_df = env.original_parameters_df


t.sleep(10)
action = env.action_modified()

action = np.array(action)
print(action)
reward = env.step_tcp_rishabh_joints_offset(action,Experiment_time=1.22)

print("reward")
print(reward)

In [ ]:
#Convert to excel
a = env.df_avg.to_numpy()
print(a[:,5])
env.df_avg.to_excel("./Original_parameters_result_tcp_euc_rishabh_joint_offset_59.xlsx")